In [54]:
#[WEB-SCRAPPING]
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import re


In [55]:
# csv resultante a partir de un primer scrapping de búsquedas en GOOGLE NEWS con dataminer. Then;

# me gustaria meter la parte de regex en otra función. cómo?
def getDf_fromCSV(file):
    df_copy = pd.read_csv(file, names = ['url','title','html','date'])
    df = df_copy.copy()
    df = df.drop(df.index[0], axis=0) #remove first (descriptions row)
    df = df.applymap(lambda x: x.lstrip())  #everyelement in each Series
    
    df[['day','month','year']] = df.date.str.split(' ', expand=True) #splitting day column into 'day','month','year
    #df[['day','month','year']] = df.date.apply(lambda x: pd.Series(str(x).split("_"))) _ otra forma de hacer lo anterior
    
    df['year'] = df["year"].replace('[a-zA-Z]','',regex =True)
    df['year'] = df["year"].replace('','2019',regex =True)
    
    df['month'] = df["month"].replace('[0-9]','',regex =True)
    
    df['month'] = df['month'].apply(lambda x: x.lower())
    df['month'] = df["month"].replace('ene.','01',regex =True)
    df['month'] = df["month"].replace('feb.','02',regex =True)
    df['month'] = df["month"].replace('mar.','03',regex =True)
    df['month'] = df["month"].replace('abr.','04',regex =True)
    df['month'] = df["month"].replace('abr.','04',regex =True)
    df['month'] = df["month"].replace('may.','05',regex =True)
    df['month'] = df["month"].replace('jun.','05',regex =True)
    df['month'] = df["month"].replace('jul.','06',regex =True)
    df['month'] = df["month"].replace('aug.','08',regex =True)
    df['month'] = df["month"].replace('sep.','09',regex =True)
    df['month'] = df["month"].replace('oct.','10',regex =True)
    df['month'] = df["month"].replace('nov.','11',regex =True)
    df['month'] = df["month"].replace('dic.','12',regex =True)
    
    df['day'] = df['day'].replace('[a-zA-Z]','',regex =True)
    
    df.drop(['date'], axis=1, inplace=True)
    df.sort_values(by=["year",'month', 'day'])
    
    df.reset_index()
    return df

df = getDf_fromCSV('crimen.csv')






In [56]:
df[0:10]

,url,title,html,day,month,year
1,https://www.abc.es/espana/madrid/abci-snoopy-s...,"«Snoopy», el sicario más sanguinario de España...",El autor del <em>crimen</em> abandonó <em>Madr...,25,05,2019
2,https://elpais.com/ccaa/2019/06/10/madrid/1560...,Un hombre mata a tiros en Aranjuez a su cuñada...,Su exmujer no estaba en el escenario del <em>c...,10,05,2019
3,https://www.laverdad.es/murcia/cartagena/crime...,El crimen de la calle San Roque,"Una hora después, cuando Petra baja a la <em>c...",7,05,2019
4,https://elpais.com/ccaa/2019/06/14/madrid/1560...,Cuando la memoria de un crimen fascista acaba ...,Cuando la memoria de un <em>crimen</em> fascis...,14,05,2019
5,https://www.libertaddigital.com/espana/2019-06...,"TVE convierte en ""desaparición"" el asesinato d...","TVE convierte en ""desaparición"" el <em>asesina...",23,05,2019
6,https://elpais.com/ccaa/2019/05/24/madrid/1558...,Asesinato de una anciana frente a la comisaría,Unos 30 pasos separan la comisaría de la Polic...,1,05,2019
7,https://www.elmundo.es/madrid/2019/06/11/5cffb...,Muere la segunda víctima del tiroteo de Aranju...,Varios testigos señalaron que el detonante del...,11,05,2019
8,https://www.elmundo.es/madrid/2019/06/14/5d02a...,"""El pecado se paga con la muerte"", el mensaje ...",El presunto autor del doble <em>crimen</em> de...,13,05,2019
9,https://www.europapress.es/cantabria/noticia-d...,El detenido por el crimen de la calle Barcelon...,El detenido por el homicidio de la <em>calle</...,13,05,2019
10,https://www.elperiodico.com/es/sociedad/201906...,Mata de un disparo a su cuñada y hiere a otras...,Los hechos tuvieron lugar en torno a las 21.50...,10,05,2019


In [57]:
# Eliminar aquellas noticias que no tengan en su html madrid, calle, avenida, etc 
# (es decir, las que no vaya a poder geolocalizar)
# para eso, entro antes en las noticias

In [64]:
def get_text_apply(url): 
    try:
        res = requests.get(url)
        # print(type(res))
        soup = BS(res.text, 'html.parser')
        parrafos = '\n'.join([p.get_text().strip() for p in soup.select("p")])
        return parrafos
    except: 
        return None

def DF_get_text_apply(df):
    df['noticia'] = df['url'].apply(get_text_apply)
    return df
df_noticias = DF_get_text_apply(df)

In [59]:
df_noticias.head()

,url,title,html,day,month,year,noticia
1,https://www.abc.es/espana/madrid/abci-snoopy-s...,"«Snoopy», el sicario más sanguinario de España...",El autor del <em>crimen</em> abandonó <em>Madr...,25,05,2019,\n\n\n\n\n\nHabían planeado el asesinato perfe...
2,https://elpais.com/ccaa/2019/06/10/madrid/1560...,Un hombre mata a tiros en Aranjuez a su cuñada...,Su exmujer no estaba en el escenario del <em>c...,10,05,2019,"El Juanín, un conocido delincuente de 38 años ..."
3,https://www.laverdad.es/murcia/cartagena/crime...,El crimen de la calle San Roque,"Una hora después, cuando Petra baja a la <em>c...",7,05,2019,\nEscribimos hoy un nuevo capítulo de la cróni...
4,https://elpais.com/ccaa/2019/06/14/madrid/1560...,Cuando la memoria de un crimen fascista acaba ...,Cuando la memoria de un <em>crimen</em> fascis...,14,05,2019,"La muerte de Arturo Ruiz, un estudiante granad..."
5,https://www.libertaddigital.com/espana/2019-06...,"TVE convierte en ""desaparición"" el asesinato d...","TVE convierte en ""desaparición"" el <em>asesina...",23,05,2019,El Telediario de la TelevisiÃ³n EspaÃ±ola de R...


In [60]:
df['noticia'].head(10)
df.head()

,url,title,html,day,month,year,noticia
1,https://www.abc.es/espana/madrid/abci-snoopy-s...,"«Snoopy», el sicario más sanguinario de España...",El autor del <em>crimen</em> abandonó <em>Madr...,25,05,2019,\n\n\n\n\n\nHabían planeado el asesinato perfe...
2,https://elpais.com/ccaa/2019/06/10/madrid/1560...,Un hombre mata a tiros en Aranjuez a su cuñada...,Su exmujer no estaba en el escenario del <em>c...,10,05,2019,"El Juanín, un conocido delincuente de 38 años ..."
3,https://www.laverdad.es/murcia/cartagena/crime...,El crimen de la calle San Roque,"Una hora después, cuando Petra baja a la <em>c...",7,05,2019,\nEscribimos hoy un nuevo capítulo de la cróni...
4,https://elpais.com/ccaa/2019/06/14/madrid/1560...,Cuando la memoria de un crimen fascista acaba ...,Cuando la memoria de un <em>crimen</em> fascis...,14,05,2019,"La muerte de Arturo Ruiz, un estudiante granad..."
5,https://www.libertaddigital.com/espana/2019-06...,"TVE convierte en ""desaparición"" el asesinato d...","TVE convierte en ""desaparición"" el <em>asesina...",23,05,2019,El Telediario de la TelevisiÃ³n EspaÃ±ola de R...



# Remove the news whose html doesnt match with "madrid", "street", "avenue"
def returnIndexes(df):
    indexes = []
    for noticia in range(len(df['noticia'])):
        print("procesando doc: " + str(noticia))
        try:
        # Guardo el índice de las filas que han hecho match en el array indexes
            if not (("calle" in df['noticia'][noticia]) or ("avenida" in df['html'][noticia]) or ("plaza" in df['noticia'][noticia])) and ("Madrid" in df['noticia'][noticia]):
                indexes.append(i)
        except:
            pass
    return indexes
indexes = (returnIndexes(df))
print(indexes)

In [61]:
def ReadDirections(file):
    df_entrada = pd.read_csv(file,sep=';', engine='python', encoding='latin1')
    df = df_entrada.copy()
    return df
df_r = ReadDirections('Coordenadas.csv')
df_r.head()

,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,VIA_NOMBRE_ACENTOS,CLASE_APP,NUMERO,CALIFICADOR,TIPO_NDP,COD_NDP,DISTRITO,BARRIO,COD_POSTAL,UTMX_ED,UTMY_ED,UTMX_ETRS,UTMY_ETRS,LATITUD,LONGITUD,ANGULO_ROTULACION
0,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,10000,EN,PARCELA,31031089,8,6,28050,"443056,33","4482503,4","442946,95","4482295,92",40°29'21.84'' N,3°40'21.84'' W,78.87
1,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,10000,SA,PARCELA,31031088,16,6,28050,"443122,46","4482490,77","443013,08","4482283,29",40°29'21.45'' N,3°40'21.45'' W,77.25
2,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,11000,EN,PARCELA,31031091,8,6,28050,"443675,22","4483309,33","443565,84","4483101,85",40°29'48.13'' N,3°39'48.13'' W,41.35
3,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,11000,SA,PARCELA,31031090,16,6,28050,"443737,55","4483251,51","443628,17","4483044,02",40°29'46.27'' N,3°39'46.27'' W,47.56
4,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,12000,EN,PARCELA,31031093,8,6,28050,"444252,07","4484081,85","444142,7","4483874,36",40°30'13.33'' N,3°39'13.33'' W,67.17


In [62]:
def CleanDataSetDirections_Coor (df):
    df['VIA_CLASE'] = df.VIA_CLASE.apply(lambda x: x if not pd.isnull(x) else '')
    df['VIA_PAR'] = df.VIA_PAR.apply(lambda x: x if not pd.isnull(x) else '')
    df['VIA_NOMBRE'] = df.VIA_NOMBRE.apply(lambda x: x if not pd.isnull(x) else '')
    dfnew = pd.DataFrame(columns=['DIRECTION'])
    dfnew['DIRECTION'] = df['VIA_CLASE'].astype(str) + ' ' + df['VIA_PAR'].astype(str) + ' ' + df['VIA_NOMBRE'].astype(str)
    df = dfnew.drop_duplicates()
    return df
df_directions = CleanDataSetDirections_Coor(df_r)
df_directions.head()

,DIRECTION
0,AUTOVÍA A-1
8,AUTOVÍA A-2
37,AUTOVÍA A-3
75,AUTOVÍA A-4
94,AUTOVÍA A-42


In [63]:
def MergeDF(df1,df2):
    df = df1[['LATITUD','LONGITUD']]
    df =  pd.merge(df,df2, left_index=True, right_index=True)
    return df
dfn = MergeDF(df_r,df_directions)
dfn.head()

,LATITUD,LONGITUD,DIRECTION
0,40°29'21.84'' N,3°40'21.84'' W,AUTOVÍA A-1
8,40°26'50.54'' N,3°39'50.54'' W,AUTOVÍA A-2
37,40°21'24.7'' N,3°33'24.7'' W,AUTOVÍA A-3
75,40°22'56.75'' N,3°41'56.75'' W,AUTOVÍA A-4
94,40°22'35.52'' N,3°43'35.52'' W,AUTOVÍA A-42


In [209]:
df_noticias = df_noticias.fillna(' ')
df_noticias.head()
display(df_noticias)

,url,title,html,day,month,year,noticia,crimen,crime,TypeOfCrime,Ocurrences
1,https://www.abc.es/espana/madrid/abci-snoopy-s...,"«Snoopy», el sicario más sanguinario de España...",El autor del <em>crimen</em> abandonó <em>Madr...,25,05,2019,\n\n\n\n\n\nHabían planeado el asesinato perfe...,detenido,,crimen,3
2,https://elpais.com/ccaa/2019/06/10/madrid/1560...,Un hombre mata a tiros en Aranjuez a su cuñada...,Su exmujer no estaba en el escenario del <em>c...,10,05,2019,"El Juanín, un conocido delincuente de 38 años ...",detenido,,crimen,3
3,https://www.laverdad.es/murcia/cartagena/crime...,El crimen de la calle San Roque,"Una hora después, cuando Petra baja a la <em>c...",7,05,2019,\nEscribimos hoy un nuevo capítulo de la cróni...,detenido,,crimen,3
4,https://elpais.com/ccaa/2019/06/14/madrid/1560...,Cuando la memoria de un crimen fascista acaba ...,Cuando la memoria de un <em>crimen</em> fascis...,14,05,2019,"La muerte de Arturo Ruiz, un estudiante granad...",detenido,,crimen,3
5,https://www.libertaddigital.com/espana/2019-06...,"TVE convierte en ""desaparición"" el asesinato d...","TVE convierte en ""desaparición"" el <em>asesina...",23,05,2019,El Telediario de la TelevisiÃ³n EspaÃ±ola de R...,detenido,,crimen,3
6,https://elpais.com/ccaa/2019/05/24/madrid/1558...,Asesinato de una anciana frente a la comisaría,Unos 30 pasos separan la comisaría de la Polic...,1,05,2019,Unos 30 pasos separan la comisaría de la Polic...,detenido,,crimen,3
7,https://www.elmundo.es/madrid/2019/06/11/5cffb...,Muere la segunda víctima del tiroteo de Aranju...,Varios testigos señalaron que el detonante del...,11,05,2019,Es la segunda víctima mortal del tiroteo ocurr...,detenido,,crimen,3
8,https://www.elmundo.es/madrid/2019/06/14/5d02a...,"""El pecado se paga con la muerte"", el mensaje ...",El presunto autor del doble <em>crimen</em> de...,13,05,2019,\nEl presunto autor del doble crimen de las do...,detenido,,crimen,3
9,https://www.europapress.es/cantabria/noticia-d...,El detenido por el crimen de la calle Barcelon...,El detenido por el homicidio de la <em>calle</...,13,05,2019,"Abonados\nSANTANDER, 13 Jun. (EUROPA PRESS) -\...",detenido,,crimen,3
10,https://www.elperiodico.com/es/sociedad/201906...,Mata de un disparo a su cuñada y hiere a otras...,Los hechos tuvieron lugar en torno a las 21.50...,10,05,2019,Iniciar sesión\nAccede con tu cuenta de Facebo...,detenido,,crimen,3


In [210]:
df_noticias.noticia.isnull().sum()
len(df_noticias.noticia)

0

In [200]:
def getTypeOfCrime(df):
    crimes = ["agresion","pelea", "peleas", "ataque",
              "sexual", "violacion","crimen","matar","asesinato","detenido"]
    lst=[]
    for noticia in df['noticia']:
        for crime in crimes:
            if crime in noticia.lower():
                lst.append(crime)
                break
    df['crime'] = lst
    return lst
crimes = (getTypeOfCrime(df_noticias))
print(crimes)

ValueError: Length of values does not match length of index

In [196]:
def getTypeOfCrime(df):
    crimes = ["crimen","agresion","pelea", "peleas", "ataque",
              "sexual", "violacion","matar","asesinato","detenido"]
    lst=[]
    for noticia in df['noticia']:
        for crime in crimes:
            if crime.lower() in noticia.lower():
                df['TypeOfCrime'] = crime
            break
        for crime in crimes:
            ocurrences = noticia.count(crime)
            df['Ocurrences'] = ocurrences
    return df
lst = (getTypeOfCrime(df_noticias))
print(lst)

                                                   url  \
1    https://www.abc.es/espana/madrid/abci-snoopy-s...   
2    https://elpais.com/ccaa/2019/06/10/madrid/1560...   
3    https://www.laverdad.es/murcia/cartagena/crime...   
4    https://elpais.com/ccaa/2019/06/14/madrid/1560...   
5    https://www.libertaddigital.com/espana/2019-06...   
6    https://elpais.com/ccaa/2019/05/24/madrid/1558...   
7    https://www.elmundo.es/madrid/2019/06/11/5cffb...   
8    https://www.elmundo.es/madrid/2019/06/14/5d02a...   
9    https://www.europapress.es/cantabria/noticia-d...   
10   https://www.elperiodico.com/es/sociedad/201906...   
11   https://www.elespanol.com/bluper/noticias/por-...   
12   https://www.abc.es/espana/madrid/abci-crimen-a...   
13   https://www.cuartopoder.es/ideas/opinion/2019/...   
14   https://www.elcomercio.es/oviedo/diez-anos-cri...   
15   https://www.abc.es/espana/aragon/abci-detenido...   
16   https://www.infolibre.es/noticias/opinion/colu...   
17   https://w

In [199]:
def getTypeOfCrime(noticia):
    crimes = ["agresion","pelea", "peleas", "ataque",
              "sexual", "violacion","crimen","matar","asesinato","detenido"]
    result = next(crime for crime in crimes if crime in noticia.lower() else: "none")
    return result
"""    result = next(crime for crime in crimes if crime in noticia.lower())
    return result"""
print(getTypeOfCrime('El crimen de la mañana que asesino sexual'))
#df['Type_crime'] = df['noticia'].apply(getTypeOfCrime)

# me cuente lo que aparece          

SyntaxError: invalid syntax (<ipython-input-199-35d66c788553>, line 4)

In [228]:
####
#     VÁLIDOOO   !!!  Me devuelve la primera
####

def getTypeOfCrime(noticia):
    crimes = ["agresion","violación","asesinato", "ataque",
              "sexual","crimen","matar","pelea","peleas","robo"]
    try: 
        noticia = noticia.lower()
        for crime in crimes: 
            if crime in noticia: 
                return crime
        else: 
            return ''
    except: 
        return ''

def DF_getTypeOfCrime(df):
    df['Type_crime'] = df['noticia'].apply(getTypeOfCrime)
    return df

In [ ]:



# cómo para que me devuelva una serie del dataframe por palabra, para ir sumando valores
# relaciones, de qué palabra sale más
# gráficas, etc




In [232]:
df[24:29]

,url,title,html,day,month,year,noticia,Type_crime
25,https://www.elconfidencial.com/espana/2019-06-...,Así funciona un jurado en Madrid: las 41 pregu...,Aunque también declararon culpables de <em>ase...,9,05,2019,El suceso se remonta a la tarde del 12 de sept...,matar
26,https://elpais.com/ccaa/2019/06/01/madrid/1559...,La policía detiene a al menos cinco aficionado...,... en un festival que se celebra con motivo d...,1,05,2019,La Policía Nacional ha detenido este sábado a ...,sexual
27,https://www.abc.es/sociedad/abci-todos-tenemos...,«Todos tenemos la culpa»: la planta de Iveco a...,... la planta de Iveco agacha la cabeza por un...,29,05,2019,"\n\n\n\n\n\nCabezas gachas. Y aflicción, una p...",sexual
28,https://www.elespanol.com/reportajes/20190605/...,"'Snoopy', el sicario culpable de matar al empr...",Un Jurado Popular de la Audiencia Provincial d...,5,05,2019,Cuenta de usuario\n\n\nPremium\nServicios\nEl ...,pelea
29,https://www.periodicodeibiza.es/pitiusas/local...,El acusado de matar a una mujer en Ibiza recon...,Al hombre se le imputa un <em>delito</em> de h...,3,05,2019,Paco S. Pérez / Europa Press | 03/06/2019\nEl ...,sexual


In [229]:

def getTypeOfCrime(df):
    crimes = ["agresion","violacion","asesinato","sexual"]
    lst=[]
    cont=0
    for noticia in df['noticia']:
        for crime in crimes:
            if crime.lower() in noticia.lower() and cont < 10:
                df['agresión'] = crime[0]
                df['violación'] = crime[1]
                df['asesinato'] = crime[2]
                df['sexual'] = crime[3]
            else:
                lst.append(' ')
            cont += 1
            if cont >= 10:
                cont =0
    return lst
lst = (getTypeOfCrime(df_noticias))
print(lst)
# de cada 10, cojo el primer elemento que no esté vacio.
print(len(lst))

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',

In [138]:
composite_list = [lst[x:x+10] for x in range(0, len(lst),10)]
#print (composite_list)
lista_2 = next(i for elemento in composite_list for i in lst if i != " ")
print(lista_2)

crimen


In [93]:
def MatchStreetsNews (df_news,df_streets):
    noticias = df_news['noticia'].str.lower()
    directorio = df_streets['DIRECTION'].str.lower()
    lst=[]
    for noticia in noticias:
        for calle in directorio:
            #print(calle)
            #print (noticia)
            if calle.lower() in noticia: 
                lst.append(calle)
            else: 
                lst.append('')
    df_news['match']=lst    
    return df_news
display(MatchStreetsNews(df_noticias,dfn))
#df_5_clean

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
